# 테크닉 51 : 물류 데이터를 불러오자

In [8]:
import pandas as pd
from pandasql import sqldf

df_of_factory = pd.read_csv("./pyda100-master/Chapter_6/tbl_factory.csv", index_col = 0)
df_of_warehouse = pd.read_csv("./pyda100-master/Chapter_6/tbl_warehouse.csv", index_col = 0)
df_of_cost = pd.read_csv("./pyda100-master/Chapter_6/rel_cost.csv", index_col = 0)
df_of_transaction = pd.read_csv("./pyda100-master/Chapter_6/tbl_transaction.csv", index_col = 0)

print("* df_of_factory :")
display(df_of_factory)
print("* df_of_warehouse :")
display(df_of_warehouse)
print("* df_of_cost :")
display(df_of_cost)
print("* df_of_transaction :")
display(df_of_transaction)

* df_of_factory :


,FCName,FCDemand,FCRegion
FCID,,,
FC00001,서울공장,28,북부
FC00002,수원공장,29,북부
FC00003,인천공장,31,북부
FC00004,대전공장,25,북부
FC00005,대구공장,21,남부
FC00006,광주공장,30,남부
FC00007,부산공장,25,남부
FC00008,울산공장,16,남부


* df_of_warehouse :


,WHName,WHSupply,WHRegion
WHID,,,
WH00001,판교창고,35,북부
WH00002,용인창고,41,북부
WH00003,수원창고,42,북부
WH00004,대전창고,60,남부
WH00005,대구창고,72,남부
WH00006,광주창고,65,남부


* df_of_cost :


,FCID,WHID,Cost
RCostID,,,
1,FC00001,WH00001,0.4
2,FC00001,WH00002,0.8
3,FC00001,WH00003,1.5
4,FC00002,WH00001,0.9
5,FC00002,WH00002,0.8
6,FC00002,WH00003,1.6
7,FC00003,WH00001,1.2
8,FC00003,WH00002,1.3
9,FC00003,WH00003,1.5


* df_of_transaction :


,TransactionDate,ToFC,FromWH,Quantity
TRID,,,,
0,2019-01-01 02:11:10,FC00004,WH00003,33
1,2019-01-01 06:12:42,FC00007,WH00006,19
2,2019-01-01 06:32:32,FC00006,WH00004,31
3,2019-01-01 07:17:06,FC00002,WH00003,18
4,2019-01-01 07:52:18,FC00001,WH00002,30
...,...,...,...,...
3995,2019-12-31 11:27:51,FC00007,WH00006,29
3996,2019-12-31 17:10:21,FC00002,WH00003,35
3997,2019-12-31 19:35:07,FC00003,WH00001,36


In [10]:
df_of_joined = sqldf("""
                 
                     SELECT A.TransactionDate,
                            A.Quantity,
                            B.Cost,
                            A.ToFC,
                            C.FCName,
                            C.FCDemand,
                            C.FCRegion,
                            A.FromWH,
                            D.WHName,
                            D.WHSupply,
                            D.WHRegion
                     FROM df_of_transaction A
                     LEFT OUTER JOIN
                     df_of_cost B
                     ON A.ToFC = B.FCID
                        AND A.FromWH = B.WHID
                     LEFT OUTER JOIN
                     df_of_factory C
                     ON A.ToFC = C.FCID
                     LEFT OUTER JOIN
                     df_of_warehouse D
                     ON A.FromWH = D.WHID
                 
                 """)
print("* df_of_joined :")
display(df_of_joined)

df_of_north = sqldf("""
                   
                       SELECT *
                       FROM   df_of_joined
                       WHERE  WHRegion = '북부'
                   
                   """)
print("* df_of_north :")
display(df_of_north)

df_of_south = sqldf("""
                   
                       SELECT *
                       FROM   df_of_joined
                       WHERE  WHRegion = '남부'
                   
                   """)
print("* df_of_south :")
display(df_of_south)

* df_of_joined :


,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FCRegion,FromWH,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,33,1.1,FC00004,대전공장,25,북부,WH00003,수원창고,42,북부
1,2019-01-01 06:12:42,19,1.3,FC00007,부산공장,25,남부,WH00006,광주창고,65,남부
2,2019-01-01 06:32:32,31,0.9,FC00006,광주공장,30,남부,WH00004,대전창고,60,남부
3,2019-01-01 07:17:06,18,1.6,FC00002,수원공장,29,북부,WH00003,수원창고,42,북부
4,2019-01-01 07:52:18,30,0.8,FC00001,서울공장,28,북부,WH00002,용인창고,41,북부
...,...,...,...,...,...,...,...,...,...,...,...
3995,2019-12-31 11:27:51,29,1.3,FC00007,부산공장,25,남부,WH00006,광주창고,65,남부
3996,2019-12-31 17:10:21,35,1.6,FC00002,수원공장,29,북부,WH00003,수원창고,42,북부
3997,2019-12-31 19:35:07,36,1.2,FC00003,인천공장,31,북부,WH00001,판교창고,35,북부
3998,2019-12-31 21:45:07,34,0.3,FC00005,대구공장,21,남부,WH00005,대구창고,72,남부


* df_of_north :


,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FCRegion,FromWH,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,33,1.1,FC00004,대전공장,25,북부,WH00003,수원창고,42,북부
1,2019-01-01 07:17:06,18,1.6,FC00002,수원공장,29,북부,WH00003,수원창고,42,북부
2,2019-01-01 07:52:18,30,0.8,FC00001,서울공장,28,북부,WH00002,용인창고,41,북부
3,2019-01-01 09:09:30,12,1.5,FC00001,서울공장,28,북부,WH00003,수원창고,42,북부
4,2019-01-01 10:52:55,27,1.5,FC00003,인천공장,31,북부,WH00003,수원창고,42,북부
...,...,...,...,...,...,...,...,...,...,...,...
1995,2019-12-30 12:29:42,22,0.9,FC00002,수원공장,29,북부,WH00001,판교창고,35,북부
1996,2019-12-30 17:51:24,32,1.3,FC00003,인천공장,31,북부,WH00002,용인창고,41,북부
1997,2019-12-31 01:50:16,33,0.8,FC00001,서울공장,28,북부,WH00002,용인창고,41,북부
1998,2019-12-31 17:10:21,35,1.6,FC00002,수원공장,29,북부,WH00003,수원창고,42,북부


* df_of_south :


,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FCRegion,FromWH,WHName,WHSupply,WHRegion
0,2019-01-01 06:12:42,19,1.3,FC00007,부산공장,25,남부,WH00006,광주창고,65,남부
1,2019-01-01 06:32:32,31,0.9,FC00006,광주공장,30,남부,WH00004,대전창고,60,남부
2,2019-01-01 08:56:09,31,0.3,FC00005,대구공장,21,남부,WH00005,대구창고,72,남부
3,2019-01-01 09:00:15,33,0.7,FC00006,광주공장,30,남부,WH00006,광주창고,65,남부
4,2019-01-01 14:12:51,21,0.7,FC00006,광주공장,30,남부,WH00006,광주창고,65,남부
...,...,...,...,...,...,...,...,...,...,...,...
1995,2019-12-31 03:35:04,20,0.7,FC00006,광주공장,30,남부,WH00006,광주창고,65,남부
1996,2019-12-31 07:05:56,11,0.8,FC00005,대구공장,21,남부,WH00004,대전창고,60,남부
1997,2019-12-31 11:27:51,29,1.3,FC00007,부산공장,25,남부,WH00006,광주창고,65,남부
1998,2019-12-31 21:45:07,34,0.3,FC00005,대구공장,21,남부,WH00005,대구창고,72,남부


# 테크닉 52 : 현재 운송량과 비용을 확인해 보자

In [19]:
# 각 지사의 비용합계 계산

print("북부지사 총비용 :", str(df_of_north["Cost"].sum()) + "만원")
print("남부지사 총비용 :", str(df_of_south["Cost"].sum()) + "만원", "\n")

# 각 지사의 운송개수 계산

print("북부지사의 총부품 운송개수 :", str(df_of_north["Quantity"].sum()) + "개")
print("남부지사의 총부품 운송개수 :", str(df_of_south["Quantity"].sum()) + "개", "\n")

# 각 지사의 부품 1개당 운송비용

price = (df_of_north["Cost"].sum() / df_of_north["Quantity"].sum()) * 10000
print("북부지사의 부품 1개당 운송비용 :", str(int(price)) + "원")
price = (df_of_south["Cost"].sum() / df_of_south["Quantity"].sum()) * 10000
print("남부지사의 부품 1개당 운송비용 :", str(int(price)) + "원", "\n")

# 각 지사별 평균 운송비용

df_of_cost_joined_with_factory = sqldf("""

                                            SELECT A.*,
                                                   B.FCRegion
                                            FROM   df_of_cost A
                                            LEFT OUTER JOIN df_of_factory B
                                            ON     A.FCID = B.FCID
                                            
                                      """)

mean_cost_of_north = sqldf("""
                           
                               SELECT AVG(Cost) AS mean_cost
                               FROM   df_of_cost_joined_with_factory
                               WHERE  FCRegion = '북부'
                           
                           """)["mean_cost"][0]
print("북부지사의 평균 운송비용 :", str(mean_cost_of_north) + "만원")

mean_cost_of_south = sqldf("""
                           
                               SELECT AVG(Cost) AS mean_cost
                               FROM   df_of_cost_joined_with_factory
                               WHERE  FCRegion = '남부'
                           
                           """)["mean_cost"][0]
print("남부지사의 평균 운송비용 :", str(mean_cost_of_south) + "만원", "\n")

print("* 남부지사가 효율 높게 부품을 운송하고 있다는 것을 알 수 있음")

북부지사 총비용 : 2189.3만원
남부지사 총비용 : 2062.0만원 

북부지사의 총부품 운송개수 : 49146개
남부지사의 총부품 운송개수 : 50214개 

북부지사의 부품 1개당 운송비용 : 445원
남부지사의 부품 1개당 운송비용 : 410원 

북부지사의 평균 운송비용 : 1.075만원
남부지사의 평균 운송비용 : 1.05만원 

* 남부지사가 효율 높게 부품을 운송하고 있다는 것을 알 수 있음


# 테크닉 53 : 네트워크를 가시화해 보자

In [7]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

df_of_weight = pd.read_csv("./pyda100-master/Chapter_6/network_weight.csv")
df_of_position = pd.read_csv("./pyda100-master/Chapter_6/network_pos.csv")

print("* df_of_weight :")
display(df_of_weight)
print("* df_of_position :")
display(df_of_position)

# 그래프 객체 생성

G = nx.Graph()

# 노드 설정

for i in range(len(df_of_weight.columns)) : 
    
    G.add_node(df_of_weight.columns[i])

# 엣지 설정

for i in range(len(df_of_weight.columns)) : 
    
    for j in range(len(df_of_weight.columns)) : 
        
        G.add_edge(df_of_weight.columns[i], df_of_weight.columns[j])
        
# 좌표 설정

pos = {}
for i in range(len(df_of_weight.columns)) : 
    
    node = df_of_weight.columns[i]
    pos[node] = (df_of_position[node][0], df_of_position[node][1])
    
# 생성된 그래프 그리기

nx.draw(G, pos, with_labels = True, font_size = 16, node_size = 1000, node_color = 'k', font_color = 'w')
plt.show()

* df_of_weight :


,A,B,C,D,E
0,0.639921,0.143353,0.944669,0.521848,0.000000
1,0.264556,0.000000,0.456150,0.568434,0.000000
2,0.617635,0.612096,0.616934,0.943748,0.000000
3,0.000000,0.000000,0.697631,0.060225,0.666767
4,0.670638,0.210383,0.000000,0.315428,0.000000


* df_of_position :


,A,B,C,D,E
0,0,0,2,2,1
1,0,2,0,2,1


TypeError: '_AxesStack' object is not callable

<Figure size 432x288 with 0 Axes>

In [6]:
df_of_weight.columns

Index(['A', 'B', 'C', 'D', 'E'], dtype='object')